In [1]:
!pip install requests
# # !pip uninstall whois
!pip install python-whois
!pip install tldextract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103246 sha256=080880f924a00a4c1cb92dab428688a81760819708ba9965a9fb99357c76d654
  Stored in directory: /root/.cache/pip/wheels/10/f1/87/145023b9a206e2e948be6480c61ef3fd3dbb81ef11b6977782
Successfully built python-whois
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 1.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import whois
import requests
import re
from urllib.parse import urlparse
from datetime import datetime
import urllib.request
import socket
from bs4 import BeautifulSoup

In [3]:
#loading the phishing URLs data to dataframe
data= pd.read_csv("dataset.csv")
data.head()

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


**Feature Extraction**

In [4]:
import features as fe
def create_vector(url,status):
  key = "wgowgcc4s4os0os8skw4wckw88s8wwkccwcsgcgg"
  flag = 1
  features = []
  response =""
  content =""
  text =""
  soup =""
  try:
    response = requests.get(url,timeout=5)
    content = response.content
    text = response.text
    soup = BeautifulSoup(response.text, 'html.parser')
  except:
    response = ""
    content =""
    text=""
    soup=""

  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    flag = 0
  features.append(url),
  features.append(status),
  features.append(fe.is_ip(url)),

  features.append(fe.length_url(url)),
  features.append(fe.length_hostname(url)),

  features.append(fe.tiny_url(url)),
  features.append(fe.countAtSign(url)),

  features.append(fe.countQuestionMark(url)),
  features.append(fe.countHyphen(url)),

  features.append(fe.countDot(url)),
  features.append(fe.countComma(url)),

  features.append(fe.countSemicolon(url)),
  features.append(fe.countDollar(url)),

  features.append(fe.countSlash(url)),
  features.append(fe.have_redirect(url)),

  features.append(fe.CountEqual(url)),
  features.append(fe.CountPercent(url)),

  features.append(fe.CountUnderScore(url)),
  features.append(fe.CountDotHostName(url)),

  features.append(fe.CountColon(url)),
  features.append(fe.CountStar(url)),

  features.append(fe.CountHttp(url)),
  features.append(fe.check_https_protocol(url)),

  features.append(fe.RatioDigitsInHostname(url)),
  features.append(fe.RatioDigitsURL(url)),

  features.append(fe.have_prefixOrSuffix(url)),
  features.append(fe.dns_expiration(url)),

  features.append(fe.web_forwarding(response)),
  features.append(fe.page_rank(key,url)),

  features.append(0 if flag==0 else fe.DomainAge(domain_name)),
  features.append(0 if flag==0 else fe.DomainRegLen(domain_name)),
  features.append(fe.LinksInScriptTags(soup,url)),
  features.append(fe.AnchorURL(soup,url)),
  features.append(fe.popup_window(content)),
  features.append(fe.abnormal_subdomain(url)),
  features.append(fe.iframe(text)),


  features.append(fe.count_www_path(url)),
  features.append(fe.count_com_path(url)),
  features.append(fe.length_word_raw(url)),

  features.append(fe.average_word_length(fe.raw_words(url))),
  features.append(fe.average_word_length(fe.raw_words_host(url))),
  features.append(fe.average_word_length(fe.raw_words_path(url))),

  features.append(fe.longest_word_length(fe.raw_words(url))),
  features.append(fe.longest_word_length(fe.raw_words_host(url))),
  features.append(fe.longest_word_length(fe.raw_words_path(url))),

  features.append(fe.shortest_word_length(fe.raw_words(url))),
  features.append(fe.shortest_word_length(fe.raw_words_host(url))),
  features.append(fe.shortest_word_length(fe.raw_words_path(url))),
  features.append(fe.web_traffic(url)),
  features.append(fe.whois_registered_domain(url))

  return features


0


In [5]:
columns =[
  'url',
  'status',
  'is_ip',
  'length_url',
  'length_hostname',
  'tiny_url',
  'CountAtSign',
  'CountQuestionMark',
  'CountHyphen',
  'CountDot',
  'CountComma',
  'CountSemicolon',
  'CountDollar',
  'CountSlash',
  'have_redirect',
  'CountEqual',
  'CountPercent',
  'CountUnderScore',
  'CountDotHostName',
  'CountColon',
  'CountStar',
  'CountHttp',
  'checkHttps',
  'RatioDigitHost',
  'RatioDigitsURL',
  'have_prefixOrSuffix',
  'dns_expiration',
  'Count_redirect',
  'page_rank',
  'domain_age',
  'domainRegLen',
  'RatioLinksTag',
  'RatioAnchorURL',
  'poppup_window',
  'abnormal_subdomain',
  'iframe',
  'Count_www_path',
  'Count_com_path',
  'length_word_raw',
  'avg_row_words',
  'avg_row_words_host',
  'avg_row_words_path',
  'longest_words_raw',
  'longest_word_host',
  'longest_word_path',
  'shortest_words_raw',
  'shortest_word_host',
  'shortest_word_path',
  'webTraffic',
  'whoisRegistered',

]

**Phishing Feature Extraction**

In [6]:
import whois
phish_features = []


for i in range(0,len(data)):
  url = data['url'][i]
  status = data['status'][i]
  phish_features.append(create_vector(url,status))



/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to conne

<ipython-input-4-7d24426fbe5c>:14: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')


Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - [Errno 111] Connection refused
Error trying to connect to socket: closing socket - [Errno 111] Connection refused
Error trying to connect to socket: closing socket - [Errno 111] Connection refused
Error trying to connect

<ipython-input-4-7d24426fbe5c>:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')


Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno 111] Connection refused
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing soc

In [7]:
phishing = pd.DataFrame(phish_features, columns= columns)
phishing.head()

,url,status,is_ip,length_url,length_hostname,tiny_url,CountAtSign,CountQuestionMark,CountHyphen,CountDot,...,avg_row_words_host,avg_row_words_path,longest_words_raw,longest_word_host,longest_word_path,shortest_words_raw,shortest_word_host,shortest_word_path,webTraffic,whoisRegistered
0,http://www.crestonwood.com/router.php,legitimate,0,37,19,0,0,0,0,3,...,7.0,4.500000,11,11,6,3,3,3,0,1
1,http://shadetreetechnology.com/V4/validation/a...,phishing,0,77,23,0,0,0,0,1,...,19.0,14.666667,32,19,32,2,19,2,0,1
2,https://support-appleld.com.secureupdate.duila...,phishing,0,126,50,0,0,1,1,4,...,8.4,8.142857,17,13,17,2,3,2,0,1
3,http://rgipt.ac.in,legitimate,0,18,11,0,0,0,0,2,...,5.0,0.000000,5,5,0,5,5,0,14064,1
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate,0,55,15,0,0,0,2,2,...,5.0,7.000000,11,7,11,3,3,4,236847,1


In [9]:
phishing.columns

Index(['url', 'status', 'is_ip', 'length_url', 'length_hostname', 'tiny_url',
       'CountAtSign', 'CountQuestionMark', 'CountHyphen', 'CountDot',
       'CountComma', 'CountSemicolon', 'CountDollar', 'CountSlash',
       'have_redirect', 'CountEqual', 'CountPercent', 'CountUnderScore',
       'CountDotHostName', 'CountColon', 'CountStar', 'CountHttp',
       'checkHttps', 'RatioDigitHost', 'RatioDigitsURL', 'have_prefixOrSuffix',
       'dns_expiration', 'Count_redirect', 'page_rank', 'domain_age',
       'domainRegLen', 'RatioLinksTag', 'RatioAnchorURL', 'poppup_window',
       'abnormal_subdomain', 'iframe', 'Count_www_path', 'Count_com_path',
       'length_word_raw', 'avg_row_words', 'avg_row_words_host',
       'avg_row_words_path', 'longest_words_raw', 'longest_word_host',
       'longest_word_path', 'shortest_words_raw', 'shortest_word_host',
       'shortest_word_path', 'webTraffic', 'whoisRegistered'],
      dtype='object')

In [8]:
file_path = 'data.csv'
phishing.to_csv(file_path, mode='a', index=False)

#Merge file